In [ ]:
#import libraries for analysis
#pandas for dataframes
import pandas as pd
#datetime for converting numerics to datetime format
import datetime as dt
#yahoo finance dividend adjusted closes for prototyping
import yfinance as yf
#statsmodels for ols regressions
import statsmodels.api as sm
#convert dataframes to numpy format because sklearn does not take dataframes well
import numpy as np
from finance_byu.fama_macbeth import fama_macbeth, fama_macbeth_parallel, fm_summary, fama_macbeth_numba


#this is the list of classifiers from sklearn that are available for analysis. "kitchen sink" approach
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split

In [ ]:
#this is a base class which we use put our parameters. All the other classes inherit from this
class BacktestParameters:
    
    def __init__(self, stk_symbols, start, end, lookback_alpha, lev):
        #the stock symbols
        self.stk_symbols = stk_symbols
        #start of the data series
        self.start = start
        #end of the data series
        self.end = end
        #index security
        #the lookback period for calculating alpha and idiosyncratic volatility
        self.lookback = lookback_alpha
        #leverage to apply to the strategy
        self.lev = lev

#class for pulling the yahoo finance data and cleaning so we have monthly data
class Data(BacktestParameters):
    
    def __init__(self, bp):
        #inherit from the base class
        self.stk_symbols = bp.stk_symbols
        self.start = bp.start
        self.end = bp.end
        self.lookback = bp.lookback
        
    #method for get getting the adjusted closes from yahoo
    def get_data(self):
        close = pd.DataFrame()

        for i in self.stk_symbols:

            df = yf.download(i, start=self.start, end=self.end, progress=False)
            df = pd.DataFrame(df['Adj Close'])
            df.columns = [i]
            if close.empty:
                close = df.copy()
            else:
                close = pd.concat([close, df], axis = 1)

        #get the daily returns
        returns = close.pct_change()[1:]

        print(returns.head())
        
        return returns
    
    #converts the daily returns to a geometric return for the month
    def convert_monthly(self, returns):
        returns = returns.resample('M').agg(lambda x: (x + 1).prod(skipna = True) - 1)
        
        #the geometric returns convert null values at the beginning of the time series to zeros.
        #this converts the data back to null values so we dont allocate to data that does not exist 
        for i in returns.columns:
            found = False
            for date, row in returns.loc[:,i].iteritems():
                if row == 0.0 and ~found:
                    returns.loc[date, i] = np.nan
                else:
                    found = True
        print(returns)
        return returns
    
    
    #this method takes a dataframe and converts to a binary output based off the center variable
    def get_binary(self, df_input, center = 0.0):
        
        binary_alpha = df_input.astype(float)
        #greater than the center var is a 1
        binary_alpha[binary_alpha >= center] = 1.0
        #less than the center var is a 0.0
        binary_alpha[binary_alpha < center] = 0.0
        
        return binary_alpha
        
#class for the backtesting of our classification
class Backtest():
    #instantiate with the backtest parameters with the returns data
    def __init__(self, bp, returns, mode):
        #the stock symbols
        self.stk_symbols = bp.stk_symbols
        #start of the data series
        self.start = bp.start
        #end of the data series
        self.end = bp.end
        #index security
        #the lookback period for calculating alpha
        self.lookback = bp.lookback
        self.returns = returns
        self.mode = mode
        
        
    def check_new_factor(self, backtest, factor_model, index_symbol):
        
        new_factor = pd.DataFrame(backtest.mean(axis = 1))
        new_factor.columns = ['portfolio']

        Y = factor_model.loc[new_factor.index, index_symbol]
        Y = pd.DataFrame(Y)
        X = factor_model.loc[new_factor.index]
        X = pd.concat([new_factor, X], axis = 1)
        X['intercept'] = 1.0
        X = X.shift(1).dropna(axis = 0)
        Y = Y.iloc[1:]
        model = sm.OLS(Y,X, missing = 'drop')
        results = model.fit()
        print(results.summary())  
        betas = results.params

        #2nd step. regress all asset returns for a fixed time period against the estimated betas to determine the risk premium for each factor.

        prems = {}
        for count, row in X.iterrows():
            y = Y.loc[count]
            y = pd.DataFrame(y).T
            x = pd.DataFrame(row).T
            model = sm.OLS(y,x, missing = 'drop')
            results = model.fit()
            prems[count] = results.params

        print("Factor Loadings")
        prems = pd.DataFrame(prems).T
        print(prems.head(20))
        expected_prems = prems.mean()
        t_stats = expected_prems / (prems.std()/ np.sqrt(len(prems.index)))
        print("expected premiums for each risk factor")
        print(expected_prems)
        print("t-stats for each risk factor")
        print(t_stats)
  
            
    """
    this method gets the equal weight portfolio of our strategy and compares to the factor model of choice. 
    Takes the backtest dataframe which is the actual returns of the strategy.
    """
    
    def factor_results(self, backtest, factor_model):
        
        port= pd.DataFrame(backtest.mean(axis = 1))
        port.columns = ['portfolio']

        #1st step. run the regression of factors on the portfolio
        Y = pd.DataFrame(port).shift(-1)
        X = factor_model.loc[port.index]
        X['intercept'] = 1.0
        model = sm.OLS(Y,X,missing = 'drop')
        results = model.fit()
        print(results.summary())  
        betas = results.params

        #2nd step. regress all asset returns for a fixed time period against the estimated betas to determine the risk premium for each factor.

        prems = {}
        for count, row in X.iterrows():
            try:
                y = Y.loc[count,:]
                y = pd.DataFrame(y).T
                x = pd.DataFrame(row).T
                model = sm.OLS(y,x, missing = 'drop')
                results = model.fit()
                prems[count] = results.params
            except:
                #this is to deal nans in the y and x
                pass

        print("Factor Loadings")
        prems = pd.DataFrame(prems).T
        print(prems.head(20))
        expected_prems = prems.mean()
        t_stats = expected_prems / (prems.std()/ np.sqrt(len(prems.index)))
        print("expected premiums for each risk factor")
        print(expected_prems)
        print("t-stats for each risk factor")
        print(t_stats)

 
    
    #this method is for actually running the backtest and getting the portfolios returns and allocations
    def run_backtest(self, input_classifiers, lookback_backtest, inputs, outputs,  ensemble = 'no', trade_type = 'both'):
        
        """
        the input_classifiers is a string of which classifier to use. This can be an array for multiple classifiers
        lookback_test is an integer for the time length to use for the feature set to use for the analysis
        inputs is a dataframe of the feature set
        output is a dataframe of the output variable to train our feature set on. This is returns across the assets
        for analysis
        ensemble is a yes or not string for whether to take an array of classifiers and a voting method to get 
        asset returns
        trade_type: this determines if the portfolio is long only, short only, or both
        """
        
        #gets the lowercase sting of the ensemble input.
        ensemble = ensemble.lower()
        
        #if we input multiple classifiers this converts the system to an ensemble output. You can only input one
        #classifier at a time or an ensemble system
        if ensemble == 'no' and len(input_classifiers) > 1:
            print("For non emsemble systems the amount of input classifiers are restricted to one. The following backtest will be changed to a ensemble system")
            ensemble = 'yes'
        
        
        #dictionary to convert the classifiers string inputs to actual sklearn classifiers
        classifiers = {"Nearest Neighbors" : KNeighborsClassifier(3), \
                       "Linear SVM" : SVC(),\
                        "RBF SVM": SVC(gamma=2, C=1), \
                        "Gaussian Process" : GaussianProcessClassifier(1.0 * RBF(1.0)), \
                        "Decision Tree" : DecisionTreeClassifier(max_depth=5), \
                        "Random Forest" : RandomForestClassifier(max_depth=3, n_estimators=2, max_features=1, random_state=1), \
                        "Neural Net": MLPClassifier(solver='adam', learning_rate = 'adaptive', alpha=1e-3, hidden_layer_sizes=(5),max_iter= 10000, random_state=1), \
                        "AdaBoost": AdaBoostClassifier(), \
                        "Naive Bayes": GaussianNB(), \
                        "QDA": QuadraticDiscriminantAnalysis()}

        #create a blank dataframe for our backtest data
        ml_backtest = pd.DataFrame()
        ml_weights = pd.DataFrame()

        #iterate through each symbol and run the backtest on the symbol
        for ticker in inputs.keys():

            count = 0

            #instantiate a dataframe for our backtest outputs for one ticker. we can only run the backtest on the index of 
            #the inputs. we remove the lookback period for analysis
            
            backtest = pd.DataFrame()
            weights = pd.DataFrame()
            
            #iterate through the inputs
            for ix, row in inputs[ticker].iterrows(): 

                if count >= lookback_backtest:
                    #the analysis df does not include the current row
                    analysis = inputs[ticker][count-lookback_backtest:count]

                    #the y var is the binary returns of the ticker
                    y = outputs[ticker].loc[analysis.index][1:].copy()
                    #the x var is the returns for the ticker on the analysis index lagged by 1
                    x = inputs[ticker].loc[analysis.index].shift(1)[1:].copy()
                    #test x represents the yesterdays x value, this is for the outsample prediction on y
                    test_x = inputs[ticker].loc[analysis.index].tail(1)
                    #to run a single classifier
                    if ensemble == 'no':
                        for name in input_classifiers:
                            #fit the classifier to the inputs
                            clf = classifiers[name]
                            clf.fit(x.to_numpy(), y.values.ravel())
                            #make a prediction on the input features 
                            predict = clf.predict(x.to_numpy())
                            predict = pd.DataFrame(data = predict, index = y.index, columns = ['predict'])
                            #mse = np.sum((y.values - predict.values.T)**2)
                            #predict the last observation in the feature set to predict y tomorrow
                            outsample = clf.predict(test_x.to_numpy())[0]
                        #if the classifier is one. we long the security times the leverage
                        if trade_type == 'both':
                            if outsample == 1.0:
                                backtest.loc[ix, ticker]  = returns.loc[ix, ticker] * lev
                                weights.loc[ix, ticker] = 1.0
                            #if the classifier is 0 we can choose to short the security or be long only
                            else:
                                backtest.loc[ix, ticker]  = -returns.loc[ix, ticker] * lev
                                weights.loc[ix, ticker] = -1.0
                                
                        elif trade_type == 'short':
                            if outsample == 1.0:
                                backtest.loc[ix, ticker]  = np.nan
                                weights.loc[ix, ticker] = np.nan
                            #if the classifier is 0 we can choose to short the security or be long only
                            else:
                                backtest.loc[ix, ticker]  = -returns.loc[ix, ticker] * lev
                                weights.loc[ix, ticker] = -1.0
                                
                        elif trade_type == 'long':
                            if outsample == 1.0:
                                backtest.loc[ix, ticker]  = returns.loc[ix, ticker] * lev
                                weights.loc[ix, ticker] = 1.0
                            #if the classifier is 0 we can choose to short the security or be long only
                            else:
                                backtest.loc[ix, ticker]  = np.nan
                                weights.loc[ix, ticker] = np.nan
                        else:
                            backtest.loc[ix, ticker]  = np.nan
                            weights.loc[ix, ticker] = np.nan
                            
                    #to do the voting method on all the classifiers
                    else:
                        all_outsample = []
                        for name in input_classifiers:
                            clf = classifiers[name]
                            clf.fit(x.to_numpy(), y.values.ravel())
                            predict = clf.predict(x.to_numpy())
                            predict = pd.DataFrame(data = predict, index = y.index, columns = ['predict'])
                            #mse = np.sum((y.values - predict.values.T)**2)
                            outsample = clf.predict(test_x.to_numpy())[0]
                            all_outsample.append(outsample)
                        #take the average of the classfier outputs 
                        avg = sum(all_outsample) / len(all_outsample)  
                        #if greater than .5, then go long the security. this paramber could be optimized
                        if avg >= 0.5:
                            backtest.loc[ix, ticker]  = returns.loc[ix, ticker] * lev
                            weights.loc[ix, ticker] = 1.0
                            
                        #if the classifier is 0 we can choose to short the security or be long only
                        else:
                            backtest.loc[ix, ticker]  = -returns.loc[ix, ticker] * lev
                            weights.loc[ix, ticker] = -1.0
                count = count + 1
                
            #concatentate the backtest for one ticker with the rest to give the portfolio. 
            if ml_backtest.empty:
                ml_backtest = backtest.copy()
                ml_weights = weights.copy()
            else:
                ml_backtest = pd.concat([ml_backtest, backtest], axis = 1)
                ml_weights = pd.concat([ml_weights, weights], axis = 1)

        #return a dataframe of all the backtests on input tickers
        return ml_backtest, ml_weights
        
        
        

In [ ]:
ff_factors = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv', index_col = 0)
ff_factors.index = pd.to_datetime(ff_factors.index, format= '%Y%m')
ff_factors = ff_factors / 100.0

factor_model = ff_factors.copy()
factor_model = factor_model[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
rf = ff_factors[['RF']]

index_symbol = 'Mkt-RF'

print(ff_factors.head(5))

In [ ]:
#start date for data
start = '2004-01-01'
#end date for data
end = '2020-01-01'

#the lookback for alpha feature
lookback = 60
#control the leverage
lev = 1.
stk_symbols  = factor_model.columns
returns = factor_model.copy()


#set the backtest parameters
bp = BacktestParameters(stk_symbols, start, end, lookback, lev  )
bt = Backtest(bp, returns, mode = 'insample')
d = Data(bp)



In [ ]:
#initialize the alpha dataframe
df_alpha = returns[lookback:].copy()
#initialize the idiosyncratic volatility as well
idiosyncratic_vol = returns[lookback:].copy()

count = 0
#iterate through the monthly data. At each point in time we find the insample alpha and idiosyncratic vol
for ix, row in returns.iterrows(): 
    #check to make sure we have at least the lookback for analysis
    if count >= lookback:
        """
        note that python will not return the last part of the index. We want to by inclusive of this
        since this is an insample measure
        """
        analysis = returns[count-lookback:count]

        for ticker in returns.columns:
                #the dependent variable is our security and index var is the independent var
            try:
                y = pd.concat([analysis[ticker], rf.loc[analysis.index]], axis = 1)
                y = pd.DataFrame(y[ticker] - y[rf.columns[0]], columns = [ticker])
                # the index is your independent variable
                x = factor_model.loc[analysis.index].drop(ticker, axis = 1)
                #add a constant so we can get the alpha
                x = sm.add_constant(x)
                x = x.shift(1)
                
                #use simple ols
                mod = sm.OLS(y, x, missing='drop')
                res = mod.fit()
                #get the alpha
                alpha = res.params['const']
                #add the results to the alpha dataframe for analysis later
                df_alpha.loc[ix, ticker] = alpha
                #the line below is in case I want to analyze a simple difference 
                #df_alpha.loc[ix, ticker] = (analysis[ticker] - analysis[index_security ]).mean()
                #find the idiosyncratic volatility. Simply the residual squared. Note this is monthly
                idiosyncratic_vol.loc[ix, ticker] = res.resid.std()
            except:
                #if we have missing data we populate our analysis dataframe with null values 
                df_alpha.loc[ix, ticker] = np.nan
                idiosyncratic_vol.loc[ix, ticker] = np.nan
    #iterate to the next point in our data
    count = count + 1

#notice that I populated the alpha and idiosyncratic volatility with the full returns dataframe. 
#these two lines remove the lookback period for clean reading
df_alpha = df_alpha.dropna(axis = 0, how = 'all')
df_alpha = df_alpha.shift(1).dropna()
idiosyncratic_vol = idiosyncratic_vol.dropna(axis = 0, how = 'all')
binary_alpha = d.get_binary(df_alpha)

print(df_alpha.head(5))
print(idiosyncratic_vol.head(5))

In [ ]:
#the next few lines are a variety of feature to use

#get the rolling means of returns as a momentum feature. Subtract the last month's return
#12 month momentum factor
rolling_mean_12 = ((returns.rolling(window = 12).mean() - returns.rolling(window = 1).mean()) )[12:]
#6 month momentum factor
rolling_mean_6 = ((returns.rolling(window = 6).mean() - returns.rolling(window = 1).mean()) )[6:]

rolling_std_12 = (returns.rolling(window = 12).std() )[12:]

#convert the momentum factor a binary variable. The center is optimized to increase strategy sharpe
#turns the momentum factor to a 1 if the mean average return over past 12 months is over 15%/12
binary_mean_12 = d.get_binary(rolling_mean_12, center = 0.0/12)
binary_mean_6 = d.get_binary(rolling_mean_6, center = 0.0/12)
#convert the output feature to a binary variable for training the classifiers
binary_returns = d.get_binary(returns, center = 0.0/12)

#creates a dictionary of different features for analysis based on ticker keys
feature_list = [binary_alpha]

feature_set = {}

for i in feature_list:
    i = i.dropna(axis = 0, how = 'all')
    for j in i.columns:
        if j not in feature_set.keys():
            i[j].rename('0', inplace = True)
            feature_set[j] = pd.DataFrame(i[j].copy())
        else:
            i[j].rename(str(len(feature_set[j].columns)), inplace = True)
            feature_set[j] = pd.concat([feature_set[j], i[j].copy()], axis = 1).dropna(axis = 0, how = 'any')
        #remove rows where we do not have features for the whole portfolio.
        feature_set[j].dropna(axis = 0, how = 'any', inplace = True)

# KNN 

In [ ]:
backtest, weights = bt.run_backtest(input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')

In [ ]:
bt.factor_results(backtest, factor_model)

In [ ]:
backtest.mean(axis = 1).cumsum().plot()

### Long only

In [ ]:
backtest, weights = bt.run_backtest(input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
bt.factor_results(backtest, factor_model)
backtest.mean(axis = 1).cumsum().plot()

### Short only

In [ ]:
backtest, weights = bt.run_backtest(input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
bt.factor_results(backtest, factor_model)
backtest.mean(axis = 1).cumsum().plot()

# Neural Net

In [ ]:
backtest, weights = bt.run_backtest(input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no')

In [ ]:
backtest.mean(axis = 1)

In [ ]:
bt.factor_results(backtest, factor_model)
backtest.mean(axis = 1).cumsum().plot()

In [ ]:
factor_model.mean(axis = 1).mean() / factor_model.mean(axis = 1).std()
backtest.mean(axis = 1).mean() / backtest.mean(axis = 1).std()

### Long only

In [ ]:
backtest, weights = bt.run_backtest(input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
bt.factor_results(backtest, factor_model)
backtest.mean(axis = 1).cumsum().plot()

### Short only

In [ ]:
backtest, weights = bt.run_backtest(input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
bt.factor_results(backtest, factor_model)
backtest.mean(axis = 1).cumsum().plot()

# Random Forest

In [ ]:
backtest, weights = bt.run_backtest(input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no')

In [ ]:
bt.factor_results(backtest, factor_model)
backtest.mean(axis = 1).cumsum().plot()

### Long Only

In [ ]:
backtest, weights = bt.run_backtest(input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
bt.factor_results(backtest, factor_model)
backtest.mean(axis = 1).cumsum().plot()

### Short Only

In [ ]:
backtest, weights = bt.run_backtest(input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
bt.factor_results(backtest, factor_model)
backtest.mean(axis = 1).cumsum().plot()

# Ensemble System

In [ ]:
backtest, weights = bt.run_backtest(input_classifiers= ["Nearest Neighbors", "Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'yes')

In [ ]:
bt.factor_results(backtest, factor_model)
backtest.mean(axis = 1).cumsum().plot()

In [ ]:
for var in factor_model.columns:
    plot_df = pd.concat([backtest[var].cumsum(), weights[var]], axis = 1)
    plot_df.columns = ['backtest', 'signal']
    plot_df.plot()
    returns.loc[weights.index, var].cumsum().plot(legend = True)

# Analyze the Market with New Factor

In [ ]:
bt.check_new_factor(backtest, factor_model, index_symbol)

# Simple Binary Decision

In [ ]:
df_alpha_shift = df_alpha.shift(1).dropna(axis = 0)
backtest = returns.loc[df_alpha_shift.index]
weights = returns.loc[df_alpha_shift.index]
for count, row in  backtest.iterrows(): 
    for j in backtest.columns:
        if df_alpha_shift.loc[count, j] >= 0:
            backtest.loc[count, j] = backtest.loc[count, j]
            weights.loc[count, j] = 1.0
        else:
            backtest.loc[count, j] = -backtest.loc[count, j]
            weights.loc[count, j] = -1.0

In [ ]:
backtest.mean(axis = 1).cumsum().plot()

In [ ]:
for var in factor_model.columns:
    plot_df = pd.concat([backtest[var].cumsum(), weights[var]], axis = 1)
    plot_df.columns = ['backtest', 'signal']
    plot_df.plot()
    returns.loc[weights.index, var].cumsum().plot(legend = True)

In [ ]:
bt.factor_results(backtest, factor_model)

In [ ]:
bt.check_new_factor(backtest, factor_model, index_symbol)